In [1]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import explode, flatten, col, concat, lit, regexp_replace, input_file_name

In [2]:
spark = SparkSession.builder.master('local[*]') \
    .appName('visualizacao') \
    .getOrCreate()

23/09/10 13:19:37 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/10 13:19:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/10 13:19:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json('data/datalake/bronze/extract_date=*/*.json')
df.printSchema()
df = df.na.drop()

root
 |-- data_extracao: string (nullable = true)
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: long (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: long (nullable = true)
 |    |    |-- sl: long (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- is: string (nullable = true)
 |    |    |    |    |-- p: long (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- sv: string (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)



In [4]:
df_p = df.select(df.data_extracao, df.hr, explode(df.l).alias('lista'))
df_p.show(5, truncate=False)
df_p.printSchema()


+----------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|data_extracao   |hr   |lista                                                                                                                                                                                                                                                                                       

In [5]:
df_p2 = df_p.select(df_p.data_extracao, df_p.lista)
df_p2.printSchema()


root
 |-- data_extracao: string (nullable = true)
 |-- lista: struct (nullable = true)
 |    |-- c: string (nullable = true)
 |    |-- cl: long (nullable = true)
 |    |-- lt0: string (nullable = true)
 |    |-- lt1: string (nullable = true)
 |    |-- qv: long (nullable = true)
 |    |-- sl: long (nullable = true)
 |    |-- vs: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |-- is: string (nullable = true)
 |    |    |    |-- p: long (nullable = true)
 |    |    |    |-- px: double (nullable = true)
 |    |    |    |-- py: double (nullable = true)
 |    |    |    |-- sv: string (nullable = true)
 |    |    |    |-- ta: string (nullable = true)



In [6]:
# df_amostra_um = df_p2.select(df_p2.data_extracao,df_p2.lista.c ,df_p2.lista.vs.p,df_p2.lista.vs.py, df_p2.lista.vs.px)
# Método para obter o dados dos ônibus
df_amostra_um = df_p2.select(
    df_p2.data_extracao,
    df_p2.lista.c.alias('LETREIRO_COMPLETO'), 
    df_p2.lista.sl.alias('SENTIDO_OPERACAO'), 
    df_p2.lista.cl.alias('CODIGO_IDENTIFICADOR_LINHA') , 
    explode(df_p2.lista.vs).alias('expansao')) \
    .select(col('data_extracao').alias('DATA_EXTRACAO'),
            'LETREIRO_COMPLETO',
            'SENTIDO_OPERACAO',
            'CODIGO_IDENTIFICADOR_LINHA',
            col('expansao.p').alias('PREFIXO_ONIBUS'),
            col('expansao.ta').alias('DATA_HORA_CAPTURA_LOCALIZACAO'),
            col('expansao.py').alias('LATITUDE') ,
            col('expansao.px').alias('LONGITUDE'),
            
            ) \
            .filter(col('expansao.p') == '15208') \
    .sort(df_p2.data_extracao.desc(),df_p2.lista.c.asc() ) \
    
df_amostra_um.show(truncate=False)


+----------------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|DATA_EXTRACAO   |LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE          |
+----------------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|2023-09-09 22:40|848L-10          |2               |33576                     |15208         |2023-09-10T01:39:54Z         |-23.4130405        |-46.750367         |
|2023-09-09 22:36|848L-10          |2               |33576                     |15208         |2023-09-10T01:35:24Z         |-23.4130405        |-46.750367         |
|2023-09-09 22:32|848L-10          |2               |33576                     |15208         |2023-09-10T01:31:38Z         |-23.4130405        |-46.750367         |
|202

In [7]:
# Método de obter operação
df_operacao =df.select(df.hr, df.data_extracao, explode('l').alias('DADOS_LINHA')) \
    .withColumnRenamed('hr', 'HORA_API') \
    .select('HORA_API',
            'DATA_EXTRACAO',
            col('DADOS_LINHA.c').alias('LETREIRO_COMPLETO'),
            col('DADOS_LINHA.cl').alias('CODIGO_IDENTIFICADOR'), 
            col('DADOS_LINHA.lt0').alias('LETREIRO_ORIGEM'),
            col('DADOS_LINHA.lt1').alias('LETREIRO_DESTINO'), 
            col('DADOS_LINHA.qv').alias('QTDE_VEICULOS_OPERACAO'),
              ).show(truncate=False)
#     .where('c == "7005-51"') \

                
   

+--------+----------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|HORA_API|DATA_EXTRACAO   |LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM      |LETREIRO_DESTINO   |QTDE_VEICULOS_OPERACAO|
+--------+----------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|11:36   |2023-09-09 11:36|6069-10          |1154                |TERM. STO. AMARO     |JD. SÃO BERNARDO   |4                     |
|11:36   |2023-09-09 11:36|6L10-41          |34773               |TERM. VARGINHA       |MESSIÂNICA         |4                     |
|11:36   |2023-09-09 11:36|2734-10          |34528               |METRÔ ITAQUERA       |JD. CAMPOS         |3                     |
|11:36   |2023-09-09 11:36|701A-10          |33457               |METRÔ VL. MADALENA   |PQ. EDU CHAVES     |8                     |
|11:36   |2023-09-09 11:36|273R-10          |32994               |METRÔ ARTU

In [13]:
df_relacao_linha = spark.read.options(header='True') \
    .csv('data/datalake/bronze/relacao_empresas_linha/*.csv')
df_relacao_linha.show(truncate=False)

23/09/10 15:44:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/relacao_empresas_1000.csv
+---+-------+-----------+-------------------------+------------------------------------------+
|_c0|LINHA  |CODIGO_AREA|CONSOCIO                 |EMPRESA                                   |
+---+-------+-----------+-------------------------+------------------------------------------+
|0  |8075-10|8          |CONSÓRCIO TRANSVIDA      |TRANSPPASS TRANSPORTE DE PASSAGEIROS LTDA.|
|1  |8075-21|8          |CONSÓRCIO TRANSVIDA      |TRANSPPASS TRANSPORTE DE PASSAGEIROS LTDA.|
|2  |8077-10|8          |CONSÓRCIO TRANSVIDA      |TRANSPPASS TRANSPORTE DE PASSAGEIROS LTDA.|
|3  |8078-10|8          |AUTO VIAÇÃO TRANSCAP LTDA|AUTO VIAÇÃO TRANSCAP LTDA           

In [14]:
df_relacao_linha.write.parquet(
    path='data/datalake/bronze/arquivo_particionado',
    mode='overwrite',
    partitionBy='CODIGO_AREA'
)

23/09/10 15:46:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/relacao_empresas_1000.csv
23/09/10 15:46:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/relacao_empresas_200.csv
23/09/10 15:46:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linh

In [15]:
df_relacao_linha.write.partitionBy('CODIGO_AREA', 'CONSOCIO').mode('overwrite').parquet('data/datalake/bronze/arquivo_particionado')

23/09/10 15:50:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/relacao_empresas_900.csv
23/09/10 15:50:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/relacao_empresas_1000.csv
23/09/10 15:50:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linh

In [21]:
df_parquet = spark.read.parquet('data/datalake/bronze/arquivo_particionado/CODIGO_AREA=2/CONSOCIO=CONSÓRCIO TRANSNOROESTE/*.parquet')
df_parquet.show(37, truncate=False)

+---+-------+----------------------------------+
|_c0|LINHA  |EMPRESA                           |
+---+-------+----------------------------------+
|3  |1016-10|NORTE BUSS TRANSPORTES S.A.       |
|5  |1018-10|NORTE BUSS TRANSPORTES S.A.       |
|26 |118Y-10|NORTE BUSS TRANSPORTES S.A.       |
|29 |1206-10|SPENCER TRANSPORTE RODOVIÁRIO LTDA|
|30 |121G-10|NORTE BUSS TRANSPORTES S.A.       |
|31 |128Y-10|NORTE BUSS TRANSPORTES S.A.       |
|37 |1701-10|NORTE BUSS TRANSPORTES S.A.       |
|38 |1701-21|NORTE BUSS TRANSPORTES S.A.       |
|39 |1701-22|NORTE BUSS TRANSPORTES S.A.       |
|40 |1702-10|SPENCER TRANSPORTE RODOVIÁRIO LTDA|
|42 |1705-10|NORTE BUSS TRANSPORTES S.A.       |
|43 |1705-51|NORTE BUSS TRANSPORTES S.A.       |
|44 |1709-10|NORTE BUSS TRANSPORTES S.A.       |
|45 |1709-21|NORTE BUSS TRANSPORTES S.A.       |
|46 |1720-10|NORTE BUSS TRANSPORTES S.A.       |
|47 |1720-21|NORTE BUSS TRANSPORTES S.A.       |
|49 |1722-10|NORTE BUSS TRANSPORTES S.A.       |
|51 |172J-10|NORTE B

In [20]:
df_parquet.count()

37